<a href="https://colab.research.google.com/github/yucan1028/thesis_code/blob/main/fuller-report/food-and-drink-report/202402/play-store-topselling-free-ranking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# リテールレポート TOPプレイヤーの顔ぶれの変化
play store ランキングの変化を見るために  
2018-10-01と2023-10-01の1ヶ月のストアランキングを算出する  
[Notion](https://www.notion.so/2023-f7846783a1914064ae24ce7f4b3ba1ca)

# Import

In [ ]:
import sys
if 'google.colab' in sys.modules:
    !pip3 install -q gcsfs

In [ ]:
import pandas as pd
from datetime import datetime
from google.cloud import storage
from google.auth import default
import gspread
import gspread_dataframe
from oauth2client.client import GoogleCredentials

# Parameters

In [ ]:
STORE_DATE = "2023-10-01"
COUNTRY = "jp"
OUTPUT_SHEET = "https://docs.google.com/spreadsheets/d/10C0xDoPDzM8jJ6osPkqDsUC-Ec_kPQuhomXQu2iKceQ/edit#gid=1894521094"
RANK_PARAMETER = 500

# constant

In [ ]:
project_id = 'fl-gerber-bigquery'

android_ranking_type = "topselling_free"
category_id = "MEDICAL"

creds, _ = default()
gc = gspread.authorize(creds)

# Method

# Auth

In [ ]:
if 'google.colab' in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

# カテゴリ取得

In [ ]:
df_apps_category = pd.io.gbq.read_gbq(
    query = f"""
    WITH
  store_info AS(
  SELECT
    timestamp,
    app_id,
    name,
  IF
    (ARRAY_LENGTH(categories)>=1,
      categories[
    OFFSET
      (0)],
      NULL) AS category,
      DATE(FORMAT_TIMESTAMP("%F %T", TIMESTAMP_SECONDS(release_date), "Asia/Tokyo")) as release_date
  FROM
    `fl-gerber-bigquery.jp_informant_v2.googleplay_store`
  WHERE
    DATE(_PARTITIONTIME) >= "{STORE_DATE}"
    AND DATE(_PARTITIONTIME) < DATE_ADD("{STORE_DATE}", INTERVAL 1 month ) ),
  category_filter AS(
  SELECT
    timestamp,
    app_id,
    name,
    category,
    release_date
  FROM
    store_info
  WHERE
    NOT category IS NULL
    ),
    latest_info AS(
  SELECT
    MAX(timestamp) AS timestamp,
    app_id
  FROM
    category_filter
  GROUP BY
    app_id),
  output_store_info AS(
  SELECT
    app_id as package_name,
    name,
    category,
    release_date
  FROM
    store_info
  JOIN
    latest_info
  USING
    (timestamp,
      app_id)
  GROUP BY
    app_id,
    name,
    category,
    release_date)
SELECT
  *
FROM
  output_store_info
""",
project_id = project_id,
)
df_apps_category.loc[df_apps_category["release_date"].isnull(), "release_date"] = "1900-01-01"
df_apps_category

,package_name,name,category,release_date
0,com.kuvileng.latestbabydollvideospudding,Baby Doll Toys Play Videos,PARENTING,1900-01-01
1,com.star5studio.toySwapPanda.diamondWitch,おもちゃのスワップパンダ,PARENTING,1900-01-01
2,com.zappotvlg,ＬＧテレビメディアプレーヤー,MEDIA_AND_VIDEO,1900-01-01
3,prokopchuk.denis.watc,My Kids on Map: Family Tracker,PARENTING,2019-03-03
4,sing.sing.play.misook,"키즈짱 씽씽동요 - 유아,어린이,동요음악",PARENTING,1900-01-01
...,...,...,...,...
907130,jp.gr.java_conf.tender.gpslocationlogger,G-Tracker- GPSロガー,MAPS_AND_NAVIGATION,2019-05-11
907131,com.skyeyes.kaohsiungVehicle,高雄市清潔資訊即時查詢,MAPS_AND_NAVIGATION,2021-03-11
907132,com.dunkapps.mobile.gpsnavigation,携帯電話番号位置GPS,MAPS_AND_NAVIGATION,2020-06-28
907133,com.tsqmadness.bmmaps,BenchMap,MAPS_AND_NAVIGATION,2013-03-12


# 現行のストアデータでのranking作成

# Get Date list

In [ ]:
DATE= f"2023-10-01"
print(DATE)
monthly_date_list = pd.date_range(DATE,pd.Period(DATE, freq="M").strftime("%Y-%m-%d"), freq='D')
monthly_date_list

2023-10-01


DatetimeIndex(['2023-10-01', '2023-10-02', '2023-10-03', '2023-10-04',
               '2023-10-05', '2023-10-06', '2023-10-07', '2023-10-08',
               '2023-10-09', '2023-10-10', '2023-10-11', '2023-10-12',
               '2023-10-13', '2023-10-14', '2023-10-15', '2023-10-16',
               '2023-10-17', '2023-10-18', '2023-10-19', '2023-10-20',
               '2023-10-21', '2023-10-22', '2023-10-23', '2023-10-24',
               '2023-10-25', '2023-10-26', '2023-10-27', '2023-10-28',
               '2023-10-29', '2023-10-30', '2023-10-31'],
              dtype='datetime64[ns]', freq='D')

In [ ]:
start_date = monthly_date_list[0].strftime("%Y-%m-%d")
end_date = monthly_date_list[-1].strftime("%Y-%m-%d")
start_date, end_date

('2023-10-01', '2023-10-31')

## google play ranking

In [ ]:
df_google_play_rank = pd.io.gbq.read_gbq(
    query = f"""
    WITH
  TABLE AS(
  SELECT
    app_id as package_name,
    rank_type,
    rank as android_rank,
    category_id,
    FORMAT_TIMESTAMP("%F", TIMESTAMP_SECONDS(timestamp), "Asia/Tokyo") AS date
  FROM
    `fl-gerber-bigquery.jp_informant_v2.googleplay_rank`
  WHERE
    DATE(_PARTITIONTIME) BETWEEN "{start_date}"
    AND "{end_date}"
    AND rank_type = "{android_ranking_type}"
    AND rank <= {RANK_PARAMETER}
    AND category_id = "{category_id}")
SELECT
  *
FROM
  TABLE
ORDER BY
  android_rank
  """,
  project_id = project_id,
)
df_google_play_rank

,package_name,rank_type,android_rank,category_id,date
0,life.medley.clinics,topselling_free,1,MEDICAL,2023-10-29
1,life.medley.clinics,topselling_free,1,MEDICAL,2023-10-23
2,life.medley.clinics,topselling_free,1,MEDICAL,2023-10-22
3,life.medley.clinics,topselling_free,1,MEDICAL,2023-10-20
4,life.medley.clinics,topselling_free,1,MEDICAL,2023-10-21
...,...,...,...,...,...
6195,com.androiddevelopermx.blogspot.organos3d,topselling_free,200,MEDICAL,2023-10-06
6196,com.widex.magnify,topselling_free,200,MEDICAL,2023-10-01
6197,com.widex.arc,topselling_free,200,MEDICAL,2023-10-15
6198,io.izn.ncms.helper,topselling_free,200,MEDICAL,2023-10-17


## ヘルスケア

In [ ]:
df_android_with_ranking = df_apps_category.merge(df_google_play_rank,how='left',on=["package_name"]).fillna(RANK_PARAMETER)
df_android_with_ranking = df_android_with_ranking[df_android_with_ranking["category"] == category_id]
df_android_with_ranking = df_android_with_ranking[["date", "package_name", "name", "category", "android_rank", "release_date"]]
df_android_with_ranking

,date,package_name,name,category,android_rank,release_date
141961,500,pl.diabdis.diabdis,Diabdis - Dzienniczek diabetyka,MEDICAL,500,1900-01-01
141962,500,com.fingerblo.odpressure,Blood Pressure Info,MEDICAL,500,2019-12-07
141963,500,jp.co.sstw.android.spp.medicalfriend2018kango,2019年版看護師国家試験問題解答・解説 メヂカルフレンド社,MEDICAL,500,1900-01-01
141964,500,com.mobincube.levothyroxine_dose_calc_without_...,Levothyroxine Dose Calculator,MEDICAL,500,2018-09-15
141965,500,com.medwiki,Med Wiki,MEDICAL,500,1900-01-01
...,...,...,...,...,...,...
155436,2023-10-29,com.medic.media.questionbank,看護師国家試験問題集クエスチョン・バンク,MEDICAL,120,2018-12-18
155437,2023-10-01,com.medic.media.questionbank,看護師国家試験問題集クエスチョン・バンク,MEDICAL,121,2018-12-18
155438,2023-10-18,com.medic.media.questionbank,看護師国家試験問題集クエスチョン・バンク,MEDICAL,123,2018-12-18
155439,2023-10-26,com.medic.media.questionbank,看護師国家試験問題集クエスチョン・バンク,MEDICAL,124,2018-12-18


### android ストアランキング

In [ ]:
df_android_average_ranking = df_android_with_ranking.groupby(["package_name", "name", "category", "release_date"],as_index=False).mean(numeric_only=True)
df_android_average_ranking = df_android_average_ranking.sort_values("android_rank").head(100)
df_android_average_ranking

,package_name,name,category,release_date,android_rank
6776,life.medley.clinics,CLINICS(クリニクス) オンライン診療・服薬指導アプリ,MEDICAL,2017-01-10,1.258065
6347,jp.epark.medicinenote,お薬手帳-病院予約もできるお薬手帳アプリ,MEDICAL,2015-09-04,2.709677
6354,jp.fastdoctor,ファストドクター｜救急の診療,MEDICAL,2019-07-28,3.774194
1091,com.arteryex.clickkarute,パシャっとカルテ-結果を写真で撮るだけでグラフ化,MEDICAL,2020-09-14,4.903226
4950,jp.ainPharmacy.anytimeAinPharmacy,いつでもアイン薬局,MEDICAL,2021-11-05,5.0
...,...,...,...,...,...
5305,jp.co.arkray.android.DiabetesSmartTool,スマートe-SMBG - 糖尿病ライフログアプリ,MEDICAL,2012-04-15,97.612903
2168,com.healint.migraineapp,頭痛ろぐ ｜頭痛記録アプリ,MEDICAL,2014-07-08,98.516129
5754,jp.co.legalprompt.TentekiA,滴下計算 点滴の滴下計算アプリ,MEDICAL,2023-07-30,99.387097
3924,com.sony.harmo,お薬の履歴を簡単管理：電子お薬手帳サービスharmo,MEDICAL,2013-11-26,99.419355


# Save Spreadsheet

In [ ]:
ss = gc.open_by_url(OUTPUT_SHEET)
gspread_dataframe.set_with_dataframe(ss.worksheet("2023-10-01メディカル無料storeランキング"), df_android_average_ranking)

# 過去のストアデータでのranking作成

# Get Date list

In [ ]:
DATE= f"2018-10-01"
print(DATE)
monthly_date_list = pd.date_range(DATE,pd.Period(DATE, freq="M").strftime("%Y-%m-%d"), freq='D')
monthly_date_list

2018-10-01


DatetimeIndex(['2018-10-01', '2018-10-02', '2018-10-03', '2018-10-04',
               '2018-10-05', '2018-10-06', '2018-10-07', '2018-10-08',
               '2018-10-09', '2018-10-10', '2018-10-11', '2018-10-12',
               '2018-10-13', '2018-10-14', '2018-10-15', '2018-10-16',
               '2018-10-17', '2018-10-18', '2018-10-19', '2018-10-20',
               '2018-10-21', '2018-10-22', '2018-10-23', '2018-10-24',
               '2018-10-25', '2018-10-26', '2018-10-27', '2018-10-28',
               '2018-10-29', '2018-10-30', '2018-10-31'],
              dtype='datetime64[ns]', freq='D')

In [ ]:
start_date = monthly_date_list[0].strftime("%Y-%m-%d")
end_date = monthly_date_list[-1].strftime("%Y-%m-%d")
start_date, end_date

('2018-10-01', '2018-10-31')

## google play ranking

In [ ]:
df_google_play_rank = pd.io.gbq.read_gbq(
    query = f"""
SELECT
  date,
  category,
  rank_type,
  rank,
  package_name
FROM
  `fl-gerber-bigquery.jp_informant.play_rank`
WHERE
  TIMESTAMP_TRUNC(_PARTITIONTIME, DAY) BETWEEN "{start_date}"
  AND "{end_date}"
  AND category = "{category_id}"
  AND rank_type = "{android_ranking_type}"
  AND rank <= {RANK_PARAMETER}
  """,
  project_id = project_id,
)
df_google_play_rank["date"] = df_google_play_rank.apply(lambda row: row["date"].strftime('%Y-%m-%d'), axis=1)
df_google_play_rank

,date,category,rank_type,rank,package_name
0,2018-10-21,MEDICAL,topselling_free,1,jp.paa.park
1,2018-10-21,MEDICAL,topselling_free,2,jp.epark.medicinenote
2,2018-10-21,MEDICAL,topselling_free,3,jp.sugi.okusuritecho
3,2018-10-21,MEDICAL,topselling_free,4,jp.co.stnet.jpamedicinehandbook
4,2018-10-21,MEDICAL,topselling_free,5,jp.co.nicho.jpokusuri
...,...,...,...,...,...
14595,2018-10-06,MEDICAL,topselling_free,496,com.comone.abcdstoma
14596,2018-10-06,MEDICAL,topselling_free,497,com.ideas.joaomeneses.snellen
14597,2018-10-06,MEDICAL,topselling_free,498,jp.rionet.rionet_remote
14598,2018-10-06,MEDICAL,topselling_free,499,jp.co.genova.app.jujoclinic.or.jp


## ヘルスケア

In [ ]:
df_android_with_ranking = df_apps_category.merge(df_google_play_rank,how='left',on=["package_name", "category"]).fillna(RANK_PARAMETER)
df_android_with_ranking = df_android_with_ranking[df_android_with_ranking["category"] == category_id]
df_android_with_ranking = df_android_with_ranking[["date", "package_name", "name", "category", "rank", "release_date"]]
df_android_with_ranking

,date,package_name,name,category,rank,release_date
141961,500,pl.diabdis.diabdis,Diabdis - Dzienniczek diabetyka,MEDICAL,500,1900-01-01
141962,500,com.fingerblo.odpressure,Blood Pressure Info,MEDICAL,500,2019-12-07
141963,2018-10-21,jp.co.sstw.android.spp.medicalfriend2018kango,2019年版看護師国家試験問題解答・解説 メヂカルフレンド社,MEDICAL,206,1900-01-01
141964,2018-10-24,jp.co.sstw.android.spp.medicalfriend2018kango,2019年版看護師国家試験問題解答・解説 メヂカルフレンド社,MEDICAL,211,1900-01-01
141965,2018-10-20,jp.co.sstw.android.spp.medicalfriend2018kango,2019年版看護師国家試験問題解答・解説 メヂカルフレンド社,MEDICAL,232,1900-01-01
...,...,...,...,...,...,...
162834,2018-10-19,com.sony.harmo.medicinenotebook2,ハルモ加盟店向けアプリ,MEDICAL,264,2016-12-14
162835,500,com.jakumonken.miepli,見え方紹介アプリ,MEDICAL,500,2019-02-01
162836,500,com.virinchi.mychat,Docquity- The Doctors' Network,MEDICAL,500,2015-09-24
162837,500,jp.co.sawai.saludi,SaluDi（サルディ）,MEDICAL,500,2021-06-30


### android ストアランキング

In [ ]:
df_android_average_ranking = df_android_with_ranking.groupby(["package_name", "name", "category", "release_date"],as_index=False).mean(numeric_only=True)
df_android_average_ranking = df_android_average_ranking.sort_values("rank").head(100)
df_android_average_ranking

,package_name,name,category,release_date,rank
6603,jp.paa.park,アイチケット - 予約で待たずに病院へ,MEDICAL,2015-08-27,1.032258
6347,jp.epark.medicinenote,お薬手帳-病院予約もできるお薬手帳アプリ,MEDICAL,2015-09-04,2.548387
6660,jp.sugi.okusuritecho,スギ薬局 おくすり手帳,MEDICAL,1900-01-01,4.096774
6005,jp.co.nicho.jpokusuri,日本調剤のお薬手帳プラス-処方箋送信・お薬情報をアプリで管理,MEDICAL,2014-09-29,5.290323
6151,jp.co.stnet.jpamedicinehandbook,日薬eお薬手帳,MEDICAL,1900-01-01,7.032258
...,...,...,...,...,...
3246,com.nttdocomo.medicine.notebook.ahd,アインお薬手帳2 ～あなたとご家族の服薬管理アプリ～,MEDICAL,1900-01-01,108.0
6089,jp.co.recruit.nurseful,看護師国家試験2880問<2019年版>予想問題付 ナースフル,MEDICAL,1900-01-01,108.387097
7084,net.stack3.bpressure,シンプル血圧手帳 誰でも簡単、印刷もできる,MEDICAL,2016-06-19,109.322581
6052,jp.co.plusr.android.ninshin_shitakamo,妊娠したかも-妊娠の可能性、初期症状をチェック-,MEDICAL,1900-01-01,110.064516


# Save Spreadsheet

In [ ]:
ss = gc.open_by_url(OUTPUT_SHEET)
gspread_dataframe.set_with_dataframe(ss.worksheet("2018-10-01メディカル無料storeランキング"), df_android_average_ranking)